In [ ]:
import pandas as pd
import sqlite3
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from dash import Dash, dcc, html, Input, Output, callback
import numpy as np

# Dash App initialisieren
app = Dash(__name__)

# Hilfsfunktion: Verfügbare Tabellen aus der Datenbank holen
def get_tables(database):
    try:
        conn = sqlite3.connect(database)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = [table[0] for table in cursor.fetchall()]
        conn.close()
        return tables
    except:
        return ['ORCL', 'MSFT', 'AAPL', 'GOOGL']  # Fallback

# Hilfsfunktion: Daten aus der Datenbank laden
def load_data(database, table, startdate=None):
    conn = sqlite3.connect(database)
    df = pd.read_sql(f'SELECT * FROM {table}', conn, index_col='Datetime')
    conn.close()
    
    if startdate is not None and startdate:
        df = df.loc[df.index >= startdate]
    
    return df

# Hilfsfunktion: KPIs berechnen
def calculate_kpis(df):
    if len(df) < 2:
        return {}
    
    latest = df.iloc[-1]
    previous = df.iloc[-2]
    
    kpis = {
        'netMargin': (latest['NetIncome'] / latest['TotalRevenue'] * 100) if latest['TotalRevenue'] != 0 else 0,
        'fcfMargin': (latest['FreeCashFlow'] / latest['TotalRevenue'] * 100) if latest['TotalRevenue'] != 0 else 0,
        'currentRatio': (latest['TotalAssets'] / latest['CurrentLiabilities']) if latest['CurrentLiabilities'] != 0 else 0,
        'roa': (latest['NetIncome'] / latest['TotalAssets'] * 100) if latest['TotalAssets'] != 0 else 0,
        'revenueGrowth': ((latest['TotalRevenue'] - previous['TotalRevenue']) / previous['TotalRevenue'] * 100) if previous['TotalRevenue'] != 0 else 0,
        'incomeGrowth': ((latest['NetIncome'] - previous['NetIncome']) / previous['NetIncome'] * 100) if previous['NetIncome'] != 0 else 0,
    }
    
    return kpis

# Layout der App
app.layout = html.Div([
    # Header
    html.Div([
        html.H1("📊 Financial Dashboard", 
               style={
                   'textAlign': 'center',
                   'color': '#00d9ff',
                   'fontWeight': 'bold',
                   'marginTop': '20px',
                   'marginBottom': '10px',
                   'fontSize': '42px'
               }),
        html.P("Comprehensive financial analysis and metrics",
              style={
                  'textAlign': 'center',
                  'color': '#9ca3af',
                  'fontSize': '16px',
                  'marginBottom': '30px'
              })
    ]),
    
    # Controls
    html.Div([
        html.Div([
            html.Label("Company:", 
                      style={'fontWeight': 'bold', 'fontSize': '14px', 'color': '#d1d5db', 'marginBottom': '8px', 'display': 'block'}),
            dcc.Dropdown(
                id='table-dropdown',
                options=[{'label': table, 'value': table} 
                        for table in get_tables('fundamentals.db')],
                value='ORCL',
                clearable=False,
                style={'backgroundColor': '#1f2937', 'color': '#000'}
            )
        ], style={'width': '30%', 'display': 'inline-block', 'marginRight': '2%', 'verticalAlign': 'top'}),
        
        html.Div([
            html.Label("Chart View:", 
                      style={'fontWeight': 'bold', 'fontSize': '14px', 'color': '#d1d5db', 'marginBottom': '8px', 'display': 'block'}),
            dcc.Dropdown(
                id='chart-view-dropdown',
                options=[
                    {'label': '📊 Overview Dashboard', 'value': 'overview'},
                    {'label': '💰 Revenue & Profitability', 'value': 'revenue'},
                    {'label': '🏦 Balance Sheet', 'value': 'balance'},
                    {'label': '📈 Margins Analysis', 'value': 'margins'},
                    {'label': '💸 Cash Flow', 'value': 'cashflow'},
                ],
                value='overview',
                clearable=False,
                style={'backgroundColor': '#1f2937', 'color': '#000'}
            )
        ], style={'width': '30%', 'display': 'inline-block', 'marginRight': '2%', 'verticalAlign': 'top'}),
        
        html.Div([
            html.Label("Start Date (optional):", 
                      style={'fontWeight': 'bold', 'fontSize': '14px', 'color': '#d1d5db', 'marginBottom': '8px', 'display': 'block'}),
            dcc.Input(
                id='startdate-input',
                type='text',
                placeholder='YYYY-MM-DD',
                style={
                    'width': '100%',
                    'padding': '8px',
                    'backgroundColor': '#1f2937',
                    'color': '#fff',
                    'border': '1px solid #374151',
                    'borderRadius': '4px',
                    'fontSize': '14px'
                }
            )
        ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ], style={'width': '90%', 'margin': '0 auto', 'marginBottom': '30px'}),
    
    # KPI Cards Container
    html.Div(id='kpi-cards', style={'width': '95%', 'margin': '0 auto', 'marginBottom': '30px'}),
    
    # Charts Container
    html.Div([
        dcc.Loading(
            id="loading",
            type="circle",
            color="#00d9ff",
            children=[
                html.Div(id='charts-container')
            ]
        )
    ], style={'width': '95%', 'margin': '0 auto'})
    
], style={
    'backgroundColor': '#0d1117',
    'minHeight': '100vh',
    'fontFamily': 'Arial, sans-serif',
    'paddingBottom': '50px'
})

# Callback für KPI Cards
@callback(
    Output('kpi-cards', 'children'),
    [Input('table-dropdown', 'value'),
     Input('startdate-input', 'value')]
)
def update_kpi_cards(selected_table, startdate):
    df = load_data('fundamentals.db', selected_table, startdate)
    kpis = calculate_kpis(df)
    
    if not kpis:
        return html.Div()
    
    card_style = {
        'backgroundColor': '#1f2937',
        'padding': '20px',
        'borderRadius': '8px',
        'border': '1px solid #374151',
        'display': 'inline-block',
        'width': '23%',
        'marginRight': '2%',
        'marginBottom': '15px',
        'verticalAlign': 'top',
        'transition': 'all 0.3s'
    }
    
    def create_kpi_card(title, value, suffix='', icon='', trend=None):
        trend_color = '#10b981' if trend and trend >= 0 else '#ef4444'
        trend_symbol = '▲' if trend and trend >= 0 else '▼'
        
        return html.Div([
            html.Div([
                html.Span(title, style={'color': '#9ca3af', 'fontSize': '14px', 'fontWeight': '500'}),
                html.Span(icon, style={'float': 'right', 'fontSize': '20px'})
            ]),
            html.Div(f"{value:.2f}{suffix}", 
                    style={'color': '#fff', 'fontSize': '28px', 'fontWeight': 'bold', 'margin': '10px 0'}),
            html.Div([
                html.Span(trend_symbol, style={'color': trend_color, 'marginRight': '5px'}) if trend else '',
                html.Span(f"{abs(trend):.2f}% vs prev" if trend else '', 
                         style={'color': trend_color, 'fontSize': '13px'})
            ]) if trend else html.Div()
        ], style=card_style)
    
    return html.Div([
        create_kpi_card('Net Margin', kpis['netMargin'], '%', '💰', kpis['incomeGrowth']),
        create_kpi_card('FCF Margin', kpis['fcfMargin'], '%', '💵'),
        create_kpi_card('Current Ratio', kpis['currentRatio'], '', '📊'),
        create_kpi_card('ROA', kpis['roa'], '%', '📈'),
        create_kpi_card('Revenue Growth', kpis['revenueGrowth'], '%', '🚀', kpis['revenueGrowth']),
        create_kpi_card('Income Growth', kpis['incomeGrowth'], '%', '💎', kpis['incomeGrowth']),
    ])

# Callback für Charts
@callback(
    Output('charts-container', 'children'),
    [Input('table-dropdown', 'value'),
     Input('chart-view-dropdown', 'value'),
     Input('startdate-input', 'value')]
)
def update_charts(selected_table, chart_view, startdate):
    df = load_data('fundamentals.db', selected_table, startdate)
    
    # Berechnungen
    netm = df['NetIncome'] / df['TotalRevenue'] * 100
    fcfm = df['FreeCashFlow'] / df['TotalRevenue'] * 100
    
    chart_config = {
        'displayModeBar': True,
        'displaylogo': False,
        'modeBarButtonsToRemove': ['pan2d', 'lasso2d', 'select2d']
    }
    
    if chart_view == 'overview':
        # 4 separate Charts im Grid
        
        # Chart 1: Revenue & Income Trend
        fig1 = go.Figure()
        fig1.add_trace(go.Bar(
            x=df['TotalRevenue'].dropna().index,
            y=df['TotalRevenue'].dropna(),
            name='Total Revenue',
            marker_color='#5352ed',
            hovertemplate='<b>Revenue</b><br>%{y:,.0f}<extra></extra>'
        ))
        fig1.add_trace(go.Bar(
            x=df['NetIncome'].dropna().index,
            y=df['NetIncome'].dropna(),
            name='Net Income',
            marker_color='#ffa502',
            hovertemplate='<b>Income</b><br>%{y:,.0f}<extra></extra>'
        ))
        fig1.add_trace(go.Scatter(
            x=df['FreeCashFlow'].dropna().index,
            y=df['FreeCashFlow'].dropna(),
            name='Free Cash Flow',
            mode='lines+markers',
            line=dict(color='#00d2d3', width=3),
            marker=dict(size=8),
            hovertemplate='<b>FCF</b><br>%{y:,.0f}<extra></extra>'
        ))
        fig1.update_layout(
            template='plotly_dark',
            title='💰 Revenue & Income Trend',
            height=400,
            hovermode='x unified',
            plot_bgcolor='#1f2937',
            paper_bgcolor='#1f2937',
            font=dict(size=12),
            showlegend=True,
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )
        
        # Chart 2: Assets vs Liabilities
        fig2 = go.Figure()
        fig2.add_trace(go.Bar(
            x=df['TotalAssets'].dropna().index,
            y=df['TotalAssets'].dropna(),
            name='Total Assets',
            marker_color='#2ed573',
            hovertemplate='<b>Assets</b><br>%{y:,.0f}<extra></extra>'
        ))
        fig2.add_trace(go.Bar(
            x=df['CurrentLiabilities'].dropna().index,
            y=df['CurrentLiabilities'].dropna(),
            name='Current Liabilities',
            marker_color='#ff4757',
            hovertemplate='<b>Liabilities</b><br>%{y:,.0f}<extra></extra>'
        ))
        fig2.update_layout(
            template='plotly_dark',
            title='🏦 Assets vs Liabilities',
            height=400,
            hovermode='x unified',
            plot_bgcolor='#1f2937',
            paper_bgcolor='#1f2937',
            font=dict(size=12),
            showlegend=True,
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )
        
        # Chart 3: Profit Margins
        fig3 = go.Figure()
        fig3.add_trace(go.Scatter(
            x=netm.dropna().index,
            y=netm.dropna(),
            name='Net Margin %',
            mode='lines+markers',
            line=dict(color='#00d2d3', width=3),
            marker=dict(size=8),
            hovertemplate='<b>Net Margin</b><br>%{y:.2f}%<extra></extra>'
        ))
        fig3.add_trace(go.Scatter(
            x=fcfm.dropna().index,
            y=fcfm.dropna(),
            name='FCF Margin %',
            mode='lines+markers',
            line=dict(color='#ff6348', width=3, dash='dash'),
            marker=dict(size=8),
            hovertemplate='<b>FCF Margin</b><br>%{y:.2f}%<extra></extra>'
        ))
        fig3.update_layout(
            template='plotly_dark',
            title='📊 Profit Margins (%)',
            height=400,
            hovermode='x unified',
            plot_bgcolor='#1f2937',
            paper_bgcolor='#1f2937',
            font=dict(size=12),
            showlegend=True,
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )
        
        # Chart 4: Growth Rates
        revenue_growth = df['TotalRevenue'].pct_change() * 100
        income_growth = df['NetIncome'].pct_change() * 100
        
        fig4 = go.Figure()
        fig4.add_trace(go.Scatter(
            x=revenue_growth.dropna().index,
            y=revenue_growth.dropna(),
            name='Revenue Growth %',
            mode='lines+markers',
            line=dict(color='#a29bfe', width=3),
            marker=dict(size=8),
            fill='tozeroy',
            hovertemplate='<b>Revenue Growth</b><br>%{y:.2f}%<extra></extra>'
        ))
        fig4.add_trace(go.Scatter(
            x=income_growth.dropna().index,
            y=income_growth.dropna(),
            name='Income Growth %',
            mode='lines+markers',
            line=dict(color='#fd79a8', width=3),
            marker=dict(size=8),
            hovertemplate='<b>Income Growth</b><br>%{y:.2f}%<extra></extra>'
        ))
        fig4.update_layout(
            template='plotly_dark',
            title='📈 Growth Rates (QoQ %)',
            height=400,
            hovermode='x unified',
            plot_bgcolor='#1f2937',
            paper_bgcolor='#1f2937',
            font=dict(size=12),
            showlegend=True,
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )
        
        return html.Div([
            html.Div([
                html.Div([dcc.Graph(figure=fig1, config=chart_config)], 
                        style={'width': '49%', 'display': 'inline-block', 'marginRight': '1%'}),
                html.Div([dcc.Graph(figure=fig2, config=chart_config)], 
                        style={'width': '49%', 'display': 'inline-block', 'marginLeft': '1%'})
            ]),
            html.Div([
                html.Div([dcc.Graph(figure=fig3, config=chart_config)], 
                        style={'width': '49%', 'display': 'inline-block', 'marginRight': '1%'}),
                html.Div([dcc.Graph(figure=fig4, config=chart_config)], 
                        style={'width': '49%', 'display': 'inline-block', 'marginLeft': '1%'})
            ])
        ])
    
    elif chart_view == 'revenue':
        fig = make_subplots(
            rows=2, cols=1,
            subplot_titles=('Revenue & Income', 'Growth Rates (%)'),
            vertical_spacing=0.15,
            row_heights=[0.6, 0.4]
        )
        
        fig.add_trace(go.Bar(
            x=df['TotalRevenue'].dropna().index,
            y=df['TotalRevenue'].dropna(),
            name='Total Revenue',
            marker_color='#5352ed',
            hovertemplate='<b>Revenue</b><br>%{y:,.0f}<extra></extra>'
        ), row=1, col=1)
        
        fig.add_trace(go.Bar(
            x=df['NetIncome'].dropna().index,
            y=df['NetIncome'].dropna(),
            name='Net Income',
            marker_color='#ffa502',
            hovertemplate='<b>Income</b><br>%{y:,.0f}<extra></extra>'
        ), row=1, col=1)
        
        fig.add_trace(go.Scatter(
            x=df['FreeCashFlow'].dropna().index,
            y=df['FreeCashFlow'].dropna(),
            name='Free Cash Flow',
            mode='lines+markers',
            line=dict(color='#00d2d3', width=3),
            hovertemplate='<b>FCF</b><br>%{y:,.0f}<extra></extra>'
        ), row=1, col=1)
        
        revenue_growth = df['TotalRevenue'].pct_change() * 100
        income_growth = df['NetIncome'].pct_change() * 100
        
        fig.add_trace(go.Scatter(
            x=revenue_growth.dropna().index,
            y=revenue_growth.dropna(),
            name='Revenue Growth %',
            mode='lines+markers',
            line=dict(color='#a29bfe', width=3),
            hovertemplate='<b>Revenue Growth</b><br>%{y:.2f}%<extra></extra>'
        ), row=2, col=1)
        
        fig.add_trace(go.Scatter(
            x=income_growth.dropna().index,
            y=income_growth.dropna(),
            name='Income Growth %',
            mode='lines+markers',
            line=dict(color='#fd79a8', width=3),
            hovertemplate='<b>Income Growth</b><br>%{y:.2f}%<extra></extra>'
        ), row=2, col=1)
        
        fig.update_layout(
            template='plotly_dark',
            title=f'<b>{selected_table}</b> - Revenue & Profitability Analysis',
            height=800,
            hovermode='x unified',
            plot_bgcolor='#1f2937',
            paper_bgcolor='#1f2937',
            showlegend=True
        )
        
        return dcc.Graph(figure=fig, config=chart_config)
    
    elif chart_view == 'balance':
        fig = make_subplots(
            rows=2, cols=1,
            subplot_titles=('Assets & Liabilities', 'Financial Ratios'),
            vertical_spacing=0.15,
            row_heights=[0.6, 0.4]
        )
        
        fig.add_trace(go.Bar(
            x=df['TotalAssets'].dropna().index,
            y=df['TotalAssets'].dropna(),
            name='Total Assets',
            marker_color='#2ed573',
            hovertemplate='<b>Assets</b><br>%{y:,.0f}<extra></extra>'
        ), row=1, col=1)
        
        fig.add_trace(go.Bar(
            x=df['CurrentLiabilities'].dropna().index,
            y=df['CurrentLiabilities'].dropna(),
            name='Current Liabilities',
            marker_color='#ff4757',
            hovertemplate='<b>Liabilities</b><br>%{y:,.0f}<extra></extra>'
        ), row=1, col=1)
        
        current_ratio = df['TotalAssets'] / df['CurrentLiabilities']
        
        fig.add_trace(go.Scatter(
            x=current_ratio.dropna().index,
            y=current_ratio.dropna(),
            name='Current Ratio',
            mode='lines+markers',
            line=dict(color='#00d2d3', width=3),
            marker=dict(size=10),
            hovertemplate='<b>Current Ratio</b><br>%{y:.2f}<extra></extra>'
        ), row=2, col=1)
        
        fig.update_layout(
            template='plotly_dark',
            title=f'<b>{selected_table}</b> - Balance Sheet Analysis',
            height=800,
            hovermode='x unified',
            plot_bgcolor='#1f2937',
            paper_bgcolor='#1f2937',
            showlegend=True
        )
        
        return dcc.Graph(figure=fig, config=chart_config)
    
    elif chart_view == 'margins':
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=netm.dropna().index,
            y=netm.dropna(),
            name='Net Margin %',
            mode='lines+markers',
            line=dict(color='#00d2d3', width=4),
            marker=dict(size=10),
            fill='tozeroy',
            hovertemplate='<b>Net Margin</b><br>%{y:.2f}%<extra></extra>'
        ))
        
        fig.add_trace(go.Scatter(
            x=fcfm.dropna().index,
            y=fcfm.dropna(),
            name='FCF Margin %',
            mode='lines+markers',
            line=dict(color='#ff6348', width=4, dash='dash'),
            marker=dict(size=10),
            hovertemplate='<b>FCF Margin</b><br>%{y:.2f}%<extra></extra>'
        ))
        
        # Operating Margin
        operating_margin = (df['NetIncome'] / df['TotalRevenue'] * 100) * 1.3  # Approximation
        fig.add_trace(go.Scatter(
            x=operating_margin.dropna().index,
            y=operating_margin.dropna(),
            name='Operating Margin % (est.)',
            mode='lines+markers',
            line=dict(color='#a29bfe', width=4),
            marker=dict(size=10),
            hovertemplate='<b>Operating Margin</b><br>%{y:.2f}%<extra></extra>'
        ))
        
        fig.update_layout(
            template='plotly_dark',
            title=f'<b>{selected_table}</b> - Margin Analysis',
            height=600,
            hovermode='x unified',
            plot_bgcolor='#1f2937',
            paper_bgcolor='#1f2937',
            showlegend=True,
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
            yaxis_title='Margin (%)',
            xaxis_title='Date'
        )
        
        return dcc.Graph(figure=fig, config=chart_config)
    
    elif chart_view == 'cashflow':
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=df['FreeCashFlow'].dropna().index,
            y=df['FreeCashFlow'].dropna(),
            name='Free Cash Flow',
            mode='lines+markers',
            line=dict(color='#00d2d3', width=4),
            marker=dict(size=10),
            fill='tozeroy',
            hovertemplate='<b>FCF</b><br>%{y:,.0f}<extra></extra>'
        ))
        
        fig.add_trace(go.Scatter(
            x=df['NetIncome'].dropna().index,
            y=df['NetIncome'].dropna(),
            name='Net Income',
            mode='lines+markers',
            line=dict(color='#ffa502', width=4, dash='dash'),
            marker=dict(size=10),
            hovertemplate='<b>Net Income</b><br>%{y:,.0f}<extra></extra>'
        ))
        
        # FCF Conversion Rate
        fcf_conversion = (df['FreeCashFlow'] / df['NetIncome'] * 100).dropna()
        
        fig.add_trace(go.Scatter(
            x=fcf_conversion.index,
            y=fcf_conversion,
            name='FCF Conversion Rate %',
            mode='lines+markers',
            line=dict(color='#a29bfe', width=3),
            marker=dict(size=8),
            yaxis='y2',
            hovertemplate='<b>FCF Conversion</b><br>%{y:.2f}%<extra></extra>'
        ))
        
        fig.update_layout(
            template='plotly_dark',
            title=f'<b>{selected_table}</b> - Cash Flow Analysis',
            height=600,
            hovermode='x unified',
            plot_bgcolor='#1f2937',
            paper_bgcolor='#1f2937',
            showlegend=True,
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
            yaxis=dict(title='Amount'),
            yaxis2=dict(title='Conversion Rate (%)', overlaying='y', side='right'),
            xaxis_title='Date'
        )
        
        return dcc.Graph(figure=fig, config=chart_config)

# App starten
if __name__ == '__main__':
    app.run()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [27/Dec/2025 08:13:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2025 08:13:55] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Dec/2025 08:13:55] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Dec/2025 08:13:55] "GET /_dash-component-suites/plotly/package_data/plotly.min.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Dec/2025 08:13:55] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2025 08:13:55] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2025 08:13:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2025 08:13:56] "POST /_dash-update-component HTTP/1.1" 200 -
